<a href="https://colab.research.google.com/github/telodigoensergio/somosnlp_reescritura_textos_administrativos/blob/main/copia_de_prueba_sin_spacy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
from urllib.parse import urljoin
import pandas as pd


In [ ]:
def scrape_text(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')

        # Remove script, style, meta, link, noscript, header, footer, aside, nav, img tags
        for tag in soup(["script", "style", "meta", "link", "noscript", "header", "footer", "aside", "nav", "img"]):
            tag.extract()

        # Elimina el contenido del mensaje de cookies
        for cookie_alert in soup.find_all("div", class_=["alertaModal", "msg cookies oculto"]):
            cookie_alert.decompose()

        # Extraer el texto manteniendo el orden de párrafos y listas
        visible_texts = []
        for element in soup.body.find_all(['p', 'li']):  # Busca párrafos y elementos de lista directamente bajo <body>
            text = element.get_text(separator=' ', strip=True)
            if text:  # Asegúrate de que el texto no esté vacío
                visible_texts.append(text)

        # Combina todos los textos extraídos en una única cadena, separando los elementos con un espacio
        visible_text = " ".join(visible_texts)

        # Limpiar el texto de espacios múltiples y saltos de línea
        visible_text = re.sub(r'\s+', ' ', visible_text)

        return visible_text.strip()

    except Exception as e:
        print(f"Error occurred while scraping the data: {e}")
        return None


In [ ]:
#Extraer links:



def extraer_links(link):
# URL base del sitio del que se extraerán los enlaces
  base_url = link

  response = requests.get(base_url)  # Haciendo la petición a la página principal
  response.raise_for_status()
  soup = BeautifulSoup(response.content, 'html.parser')

  links = set()  # Utiliza un conjunto para evitar duplicados
  for link in soup.find_all('a'):
      href = link.get('href')
      if href and not href.startswith('#'):  # Ignorar enlaces internos de la página que comienzan con '#'
        full_url = urljoin(base_url, href)  # Convierte rutas relativas en absolutas usando la URL base
        links.add(full_url)
  df_enlaces = pd.DataFrame(list(links), columns=['Enlaces'])

  return df_enlaces

In [ ]:
df_enlaces = extraer_links("https://www.hacienda.gob.es/es-ES/Paginas/Home.aspx")


df_enlaces['texto'] = df_enlaces['Enlaces'].apply(scrape_text)

Error occurred while scraping the data: 'NoneType' object has no attribute 'find_all'


Error occurred while scraping the data: 'NoneType' object has no attribute 'find_all'


Error occurred while scraping the data: 'NoneType' object has no attribute 'find_all'
Error occurred while scraping the data: 'NoneType' object has no attribute 'find_all'


Error occurred while scraping the data: 'NoneType' object has no attribute 'find_all'


Error occurred while scraping the data: 'NoneType' object has no attribute 'find_all'


...

Error occurred while scraping the data: 'NoneType' object has no attribute 'find_all'


Error occurred while scraping the data: 'NoneType' object has no attribute 'find_all'


Error occurred while scraping the data: 'NoneType' object has no attribute 'find_all'


Error occurred while scraping the data: 'NoneType' object has no attribute 'find_all'


Error occurred while scraping the data: 'NoneType' object has no attribute 'find_all'
Error occurred while scraping the data: 403 Client Error: Forbidden for url: https://administracion.gob.es/
Error occurred while scraping the data: 403 Client Error: Forbidden for url: https://transparencia.gob.es/


In [ ]:
!pip install groq

from groq import Groq
from google.colab import userdata

client = Groq(
    api_key=userdata.get("GROQ_API_KEY"),
)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.3 MB/s eta 0:00:00


In [ ]:
def clarify_text (text):
  prompt = """ You are an expert in clear language in Spanish who turns any text from the web into accessible and easy to understand for 12-year-old children.
  The original text is about a specific topic (for example, an administrative procedure, a description of a service, etc.) and
  I want it to be reformulated following clear language guidelines. Here are the guidelines I would like you to follow:

1. Remove non-essential elements such as references to social networks, site navigation (menu tags, internal links, etc.), and any other content that does not contribute to the main information.

2. Simplify the vocabulary: Use simple words and avoid technical terms or specialized jargon. When it is necessary to use a technical term, include a brief explanation in simple terms.

3. Organize the information into clear points or steps THIS IS VERY IMPORTANT!: This helps to make the text easier to follow. Use numbers for important steps or points.

4. Avoid long and complicated sentences: Try to keep sentences short and to the point. One idea per sentence.

5. Use the active voice instead of the passive: This makes the text more direct and easier to read.

The goal is for the text to be understandable for teenagers aged 14, people with a basic level of Spanish, and those with a low degree of literacy.

HIGHLY IMPORTANT: YOU CAN ONLY ANSWER IN SPANISH!!

The text is as follows:
"""
  if text is None:
    return " "

  if text is not None:
    input = prompt + text
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": input,
                }
    ],
        model="mixtral-8x7b-32768",
        temperature = 1.15
        )
    clarified_text = (chat_completion.choices[0].message.content)
  return clarified_text


In [ ]:
df_enlaces.loc[:20, 'texto limpio'] = df_enlaces['texto'].iloc[:21].apply(clarify_text)

In [ ]:
df_enlaces.to_csv('datset.csv')

##TERRITORIO DE PRUEBAS

In [ ]:
import pprint
url_getafe = "https://sede.getafe.es/sta/CarpetaPublic/doEvent?APP_CODE=STA&PAGE_CODE=CATALOGO&DETALLE=6269000961339052507583"
texto_scrap_getafe = scrape_text(url_getafe)

pprint.pprint(texto_scrap_getafe)



('El volante/certificado de empadronamiento es un documento de carácter '
 'informativo en relación a la acreditación de residencia y domicilio habitual '
 'en el municipio, a efectos administrativos. El volante de empadronamiento y '
 'el certificado de empadronamiento pueden ser: Individual: donde sólo constan '
 'los datos del solicitante. Familiar o Colectivo: donde constan todas las '
 'personas empadronadas en ese domicilio. Histórico Individual o Colectivo: '
 'donde consta de forma individual o colectiva el tiempo empadronado en el '
 'domicilio. LOS VOLANTES/CERTIFICADOS INDIVIDUALES puede solicitarse, además '
 'de presencialmente, online del siguiente modo: Con certificado electrónico: '
 'a través de la opción de PADRON DE HABITANTES de " Mi Carpeta Ciudadana " y '
 'podrá descargarlo inmediatamente. Sin certificado electrónico: a través de '
 'la Oficina Virtual puede solicitar volante/certificado individual que le '
 'será remitido por correo ordinario a la dirección en l

In [ ]:
texto_sin_aclarar = """El volante/certificado de empadronamiento es un documento de carácter informativo en
relación a la acreditación de residencia y domicilio habitual en el municipio, a efectos administrativos.
El volante de empadronamiento y el certificado de empadronamiento pueden ser: Individual: donde sólo constan
los datos del solicitante. Familiar o Colectivo: donde constan todas las personas empadronadas en ese domicilio.
Histórico Individual o Colectivo: donde consta de forma individual o colectiva el tiempo empadronado en el domicilio.
LOS VOLANTES/CERTIFICADOS INDIVIDUALES puede solicitarse, además de presencialmente, online del siguiente modo:
 Con certificado electrónico: a través de la opción de PADRON DE HABITANTES de "Mi Carpeta Ciudadana" y podrá descargarlo inmediatamente.
  Sin certificado electrónico: a través de la Oficina Virtual puede solicitar volante/certificado individual
   que le será remitido por correo ordinario a la dirección en la que Vd. figure empadronado."""

texto_getafe_aclarado = clarify_text(texto_sin_aclarar)
print(texto_getafe_aclarado)

El volante o certificado de empadronamiento es un documento que da información sobre dónde vives de forma habitual para fines administrativos. Puede ser individual, con solo tus datos, o familiar/colectivo, con los datos de todas las personas que viven en tu domicilio. También puede ser histórico, mostrando el tiempo que has vivido en esa dirección.

El volante o certificado individual se puede pedir de dos formas:

1. Con un certificado electrónico: Puedes hacerlo a través de "Mi Carpeta Ciudadana" y obtenerlo de inmediato.
2. Sin certificado electrónico: Puedes solicitarlo en la Oficina Virtual y te lo enviarán por correo postal a tu dirección de residencia.

Recuerda que el volante o certificado histórico individual o colectivo solo se puede pedir de forma presencial.
